# util.py

In [1]:
# Feel free to modify this to your needs. We will not rely on your util.py
from os import path

# If you want this to work do not move this file
# SRC_PATH = path.dirname(path.abspath(__file__))
# DATA_PATH = path.join(SRC_PATH, '..', 'data')



YEARBOOK_PATH = path.join(DATA_PATH, "yearbook", "yearbook")
STREETVIEW_PATH = path.join(DATA_PATH, "geo", "geo")

import re
yb_r = re.compile("(\d\d\d\d)_(.*)_(.*)_(.*)_(.*)")
sv_r = re.compile("([+-]?\d*\.\d*)_([+-]?\d*\.\d*)_\d*_-004")

# Get the label for a file
# For yearbook this returns a year
# For streetview this returns a (longitude, latitude) pair
def label(filename):
  m = yb_r.search(filename)
  if m is not None: return int(m.group(1))
  m = sv_r.search(filename)
  assert m is not None, "Filename '%s' malformatted"%filename
  return float(m.group(2)), float(m.group(1))

# List all the yearbook files:
#   train=True, valid=False will only list training files (for training)
#   train=False, valid=True will only list validation files (for testing)
def listYearbook(train=True, valid=True):
  r = []
  if train: r = r + [n.strip().split('\t') for n in open(YEARBOOK_PATH+'_train.txt','r')]
  if valid: r = r + [n.strip().split('\t') for n in open(YEARBOOK_PATH+'_valid.txt','r')]
  return r

# List all the streetview files
def listStreetView(train=True, valid=True):
  r = []
  if train: r = r + [n.strip().split('\t') for n in open(STREETVIEW_PATH+'_train.txt','r')]
  if valid: r = r + [n.strip().split('\t') for n in open(STREETVIEW_PATH+'_valid.txt','r')]
  return r

def testListYearbook():
  r = []
  r = r + [n.strip().split('\t') for n in open(YEARBOOK_PATH+'_test.txt','r')]
  return r

def testListStreetView():
  r = []
  r = r + [n.strip().split('\t') for n in open(STREETVIEW_PATH+'_test.txt','r')]
  return r

try:
  from mpl_toolkits.basemap import Basemap
  basemap_params = dict(projection='merc',llcrnrlat=40.390225,urcrnrlat=52.101005, llcrnrlon=-5.786422,urcrnrlon=10.540445, resolution='l')
  BM = Basemap(**basemap_params)
except:
  BM = None

# Draw some coordinates for geolocation
# This function expects a 2d numpy array (N, 2) with latutudes and longitudes in them
def drawOnMap(coordinates):
  from pylab import scatter
  import matplotlib.pyplot as plt
  assert BM is not None, "Failed to load basemap. Consider running `pip install basemap`."
  BM.drawcoastlines()
  BM.drawcountries()
  # This function expects longitude, latitude as arguments
  x,y = BM(coordinates[:,0], coordinates[:,1])
  scatter( x, y )
  plt.show()

# Map coordinates to XY positions (useful to compute distances)
# This function expects a 2d numpy array (N, 2) with latitudes and longitudes in them
def coordinateToXY(coordinates):
  import numpy as np
  assert BM is not None, "Failed to load basemap. Consider running `pip install basemap`."
  return np.vstack( BM(coordinates[:,0], coordinates[:,1]) ).T

# inverse of the above
def XYToCoordinate(xy):
  import numpy as np
  assert BM is not None, "Failed to load basemap. Consider running `pip install basemap`."
  return np.vstack( BM(xy[:,0], xy[:,1], inverse=True) ).T



NameError: name 'DATA_PATH' is not defined

# grade.py

In [2]:
from os import path
import util
import numpy as np
import argparse
from skimage.io import imread
from util import *
import csv

def load(image_path):
	#TODO:load image and process if you want to do any
	img=imread(image_path)
	return img
	
class Predictor:
	DATASET_TYPE = 'yearbook'
	# baseline 1 which calculates the median of the train data and return each time
	def yearbook_baseline(self):
		# Load all training data
		train_list = listYearbook(train=True, valid=False)

		# Get all the labels
		years = np.array([float(y[1]) for y in train_list])
		med = np.median(years, axis=0)
		return [med]

	# Compute the median.
	# We do this in the projective space of the map instead of longitude/latitude,
	# as France is almost flat and euclidean distances in the projective space are
	# close enough to spherical distances.
	def streetview_baseline(self):
		# Load all training data
		train_list = listStreetView(train=True, valid=False)

		# Get all the labels
		coord = np.array([(float(y[1]), float(y[2])) for y in train_list])
		xy = coordinateToXY(coord)
		med = np.median(xy, axis=0, keepdims=True)
		med_coord = np.squeeze(XYToCoordinate(med))
		return med_coord

	def predict(self, image_path):

		img = load(image_path)

		#TODO: load model

		#TODO: predict model and return result either in geolocation format or yearbook format
		# depending on the dataset you are using
		if self.DATASET_TYPE == 'geolocation':
			result = self.streetview_baseline() #for geolocation
		elif self.DATASET_TYPE == 'yearbook':
			result = self.yearbook_baseline() #for yearbook
		return result
		
	




ImportError: No module named 'skimage'

# run.py

In [3]:
from __future__ import print_function
from os import path
from math import sin, cos, atan2, sqrt, pi
from run import *
from util import *

SRC_PATH = path.dirname(path.abspath(__file__))
DATA_PATH = path.join(SRC_PATH, '..', 'data')
YEARBOOK_PATH = path.join(DATA_PATH, 'yearbook')
YEARBOOK_VALID_PATH = path.join(YEARBOOK_PATH, 'valid')
YEARBOOK_TEST_PATH = path.join(YEARBOOK_PATH, 'test')
YEARBOOK_TEST_LABEL_PATH = path.join(SRC_PATH, '..', 'output', 'yearbook_test_label.txt')
STREETVIEW_PATH = path.join(DATA_PATH, 'geo')
STREETVIEW_VALID_PATH = path.join(STREETVIEW_PATH, 'valid')
STREETVIEW_TEST_PATH = path.join(STREETVIEW_PATH, 'test')
STREETVIEW_TEST_LABEL_PATH = path.join(SRC_PATH, '..', 'output', 'geo_test_label.txt')

def numToRadians(x):
  return x / 180.0 * pi

# Calculate L1 score: max, min, mean, standard deviation
def maxScore(ground_truth, predicted_values):
  diff = np.absolute(np.array(ground_truth) - np.array(predicted_values))
  diff_sum = np.sum(diff, axis = 1)
  return np.max(diff_sum), np.min(diff_sum), np.mean(diff_sum), np.std(diff_sum)

# Calculate distance (km) between Latitude/Longitude points
# Reference: http://www.movable-type.co.uk/scripts/latlong.html
EARTH_RADIUS = 6371
def dist(lat1, lon1, lat2, lon2):
  lat1 = numToRadians(lat1)
  lon1 = numToRadians(lon1)
  lat2 = numToRadians(lat2)
  lon2 = numToRadians(lon2)

  dlat = lat2 - lat1
  dlon = lon2 - lon1

  a = sin(dlat / 2.0) * sin(dlat / 2.0) + cos(lat1) * cos(lat2) * sin(dlon / 2.0) * sin(dlon / 2.0)
  c = 2 * atan2(sqrt(a), sqrt(1-a))

  d = EARTH_RADIUS * c
  return d

# Evaluate L1 distance on valid data for yearbook dataset
def evaluateYearbook(Predictor):
  test_list = util.listYearbook(False, True)
  predictor = Predictor()
  predictor.DATASET_TYPE = 'yearbook'

  total_count = len(test_list)
  l1_dist = 0.0
  print( "Total validation data", total_count )
  count=0
  for image_gr_truth in test_list:
    image_path = path.join(YEARBOOK_VALID_PATH, image_gr_truth[0])
    pred_year = predictor.predict(image_path)
    truth_year = int(image_gr_truth[1])
    l1_dist += abs(pred_year[0] - truth_year)
    count=count+1

  l1_dist /= total_count
  print( "L1 distance", l1_dist)
  return l1_dist

# Evaluate L1 distance on valid data for geolocation dataset
def evaluateStreetview(Predictor):
  test_list = listStreetView(False, True)
  predictor = Predictor()
  predictor.DATASET_TYPE = 'geolocation'

  total_count = len(test_list)
  l1_dist = 0
  print( "Total validation data", total_count )
  for image_gr_truth in test_list:
    image_path = path.join(STREETVIEW_VALID_PATH, image_gr_truth[0])
    pred_lat, pred_lon = predictor.predict(image_path)
    truth_lat, truth_lon = float(image_gr_truth[1]), float(image_gr_truth[2])
    l1_dist += dist(pred_lat, pred_lon, truth_lat, truth_lon)
  l1_dist /= total_count
  print( "L1 distance", l1_dist )
  return l1_dist

# Predict label for test data on yearbook dataset
def predictTestYearbook(Predictor):
  test_list = util.testListYearbook()
  predictor = Predictor()
  predictor.DATASET_TYPE = 'yearbook'

  total_count = len(test_list)
  print( "Total test data: ", total_count )

  output = open(YEARBOOK_TEST_LABEL_PATH, 'w')
  for image in test_list:
    image_path = path.join(YEARBOOK_TEST_PATH, image[0])
    pred_year = predictor.predict(image_path)
    out_string = image[0] + '\t' + str(pred_year[0]) + '\n'
    output.write(out_string)
  output.close()

# Predict label for test data for geolocation dataset
def predictTestStreetview(Predictor):
  test_list = testListStreetView()
  predictor = Predictor()
  predictor.DATASET_TYPE = 'geolocation'

  total_count = len(test_list)
  print( "Total test data", total_count)

  output = open(STREETVIEW_TEST_LABEL_PATH, 'w')
  for image in test_list:
    image_path = path.join(STREETVIEW_TEST_PATH, image[0])
    pred_lat, pred_lon = predictor.predict(image_path)
    out_string = image[0] + '\t' + str(pred_lat) + '\t' + str(pred_lon) + '\n'
    output.write(out_string)
  output.close()

if __name__ == "__main__":
  import importlib
  from argparse import ArgumentParser
  parser = ArgumentParser("Evaluate a model on the validation set")
  parser.add_argument("--DATASET_TYPE", dest="dataset_type",
                      help="Dataset: yearbook/geolocation", required=True)
  parser.add_argument("--type", dest="type",
                      help="Dataset: valid/test", required=True)

  args = parser.parse_args()
  if args.dataset_type == 'yearbook':
    print("Yearbook")
    if (args.type == 'valid'):
      evaluateYearbook(Predictor)
    elif (args.type == 'test'):
      predictTestYearbook(Predictor)
    else:
      print("Unknown type '%s'", args.type)
  elif args.dataset_type == 'geolocation':
    print("Geolocation")
    if (args.type == 'valid'):
      evaluateStreetview(Predictor)
    elif (args.type == 'test'):
      predictTestStreetview(Predictor)
    else:
      print("Unknown type '%s'", args.type)
  else:
    print("Unknown dataset type '%s'", Predictor.DATASET_TYPE)
    exit(1)
    

ImportError: No module named 'skimage'